# Translating Tweets

In [ ]:
!git clone https://github.com/cardiffnlp/xlm-t

In [ ]:
import os
import pandas as pd

def process_language(language):
    base_dir = "/content/xlm-t/data/sentiment"
    lang_path = os.path.join(base_dir, language)

    all_rows = []

    for split in ["train", "val", "test"]:
        text_file_path = os.path.join(lang_path, f"{split}_text.txt")
        label_file_path = os.path.join(lang_path, f"{split}_labels.txt")

        if not os.path.exists(text_file_path) or not os.path.exists(label_file_path):
            print(f"Missing files for split '{split}' in {language}, skipping.")
            continue

        with open(text_file_path, encoding="utf-8") as text_file, \
             open(label_file_path, encoding="utf-8") as label_file:
            texts = text_file.read().splitlines()
            labels = label_file.read().splitlines()

        for text, label in zip(texts, labels):
            all_rows.append({
                "original_text": text,
                "label": int(label),
                "Split": split
            })

    df = pd.DataFrame(all_rows)
    output_path = os.path.join(base_dir, f"{language}.csv")
    df.to_csv(output_path, index=False, encoding="utf-8")
    print(f"Saved combined dataset to: {output_path}")

In [ ]:
process_language("hindi")
process_language("arabic")
process_language("french")
process_language("german")
process_language("italian")
process_language("portuguese")
process_language("spanish")

In [ ]:
import csv
import time
import random
import os
from openai import AzureOpenAI

# API setup
os.environ["AI_SANDBOX_KEY"] = "6662b94a6403410284b9271b8f2afa4c"
sandbox_api_key = os.environ.get('AI_SANDBOX_KEY')

sandbox_endpoint = "https://api-ai-sandbox.princeton.edu/"
sandbox_api_version = "2025-03-01-preview"

client = AzureOpenAI(
    api_key=sandbox_api_key,
    azure_endpoint=sandbox_endpoint,
    api_version=sandbox_api_version
)

prompt = '''
Translate the text below to English. Keep the translation as close to the original in tone and style as you can.
'''

csv_dir = "/content/xlm-t/data/sentiment"
output_dir = os.path.join(csv_dir, "translated")
os.makedirs(output_dir, exist_ok=True)

def translate(language):
    input_path = os.path.join(csv_dir, f"{language}.csv")
    output_path = os.path.join(output_dir, f"{language}_gpt4o_translations.csv")

    with open(input_path, newline='', encoding="utf-8") as infile, \
         open(output_path, "w", newline='', encoding="utf-8") as outfile:

        reader = csv.DictReader(infile)
        fieldnames = ["row_index", "original_text", "translated_text", "label", "Split"]
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()

        for idx, row in enumerate(reader):
            original_text = row["original_text"]
            label = row["label"]
            split = row.get("Split", "unspecified")  # default to "unspecified" if missing
            full_prompt = f"{prompt}\n\n{original_text}"

            try:
                response = client.chat.completions.create(
                    model="gpt-4o",
                    temperature=0,
                    max_tokens=500,
                    top_p=0.1,
                    messages=[
                        {"role": "system", "content": "You are a helpful assistant."},
                        {"role": "user", "content": full_prompt},
                    ]
                )

                translated_text = response.choices[0].message.content.strip()

                writer.writerow({
                    "row_index": idx,
                    "original_text": original_text,
                    "translated_text": translated_text,
                    "label": label,
                    "Split": split
                })

            except Exception as e:
                print(f"Error on row {idx}: {e}")
                writer.writerow({
                    "row_index": idx,
                    "original_text": original_text,
                    "translated_text": f"ERROR: {e}",
                    "label": label,
                    "Split": split
                })

            time.sleep(0.3)

if __name__ == "__main__":
    languages = ["portuguese", "hindi"]
    for lang in languages:
        translate(lang)
        print(f"Done with {lang}")
    print("Done!")

# Finetuning Sentiment Analysis Model

In [ ]:
!pip install datasets

In [ ]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import shutil

In [ ]:
def preprocess_data(df):
    df = df[['translated_text', 'label', 'Split']].rename(columns={'translated_text': 'text'})
    df = df.dropna()
    df = df[~df['text'].str.startswith("ERROR:")]  # skip translation errors
    df = df[df['label'].isin([0, 1, 2])]
    return df

def tokenize_function(example, tokenizer):
    return tokenizer(example['text'], truncation=True, padding='max_length', max_length=128)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = predictions.argmax(axis=-1)
    acc = accuracy_score(labels, preds)
    report = classification_report(labels, preds, output_dict=True)
    return {
        'accuracy': acc,
        'precision': report['weighted avg']['precision'],
        'recall': report['weighted avg']['recall'],
        'f1': report['weighted avg']['f1-score']
    }

def plot_confusion_matrix(cm, class_names):
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.tight_layout()
    return plt

In [ ]:
def finetune_sentiment_model(language):
    print(f"\nProcessing language: {language}")
    base_dir = "/content/xlm-t/data/sentiment/translated"
    df = pd.read_csv(f"{base_dir}/{language}_gpt4o_translations.csv")

    model_name = 'cardiffnlp/twitter-roberta-base-sentiment'
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    class_names = ["Negative", "Neutral", "Positive"]

    data = preprocess_data(df)

    train_df = data[data['Split'] == 'train']
    val_df = data[data['Split'] == 'val']
    test_df = data[data['Split'] == 'test']

    train_ds = Dataset.from_pandas(train_df[['text', 'label']])
    val_ds = Dataset.from_pandas(val_df[['text', 'label']])
    test_ds = Dataset.from_pandas(test_df[['text', 'label']])

    train_ds = train_ds.map(lambda x: tokenize_function(x, tokenizer), batched=True)
    val_ds = val_ds.map(lambda x: tokenize_function(x, tokenizer), batched=True)
    test_ds = test_ds.map(lambda x: tokenize_function(x, tokenizer), batched=True)

    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

    training_args = TrainingArguments(
        output_dir=f"./results_{language}_gpt4o_temp",
        eval_strategy="epoch",
        save_strategy="no",
        learning_rate=3e-5,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=15,
        weight_decay=0.1,
        logging_steps=10,
        logging_dir=f"./logs_{language}_gpt4o",
        report_to="none",
        metric_for_best_model="eval_accuracy",
        warmup_ratio=0.1,
    )

    early_stopping_callback = EarlyStoppingCallback(
        early_stopping_patience=2,
        early_stopping_threshold=0.001
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[early_stopping_callback]
    )

    trainer.train()

    print("Evaluating on test set...")
    test_metrics = trainer.evaluate(test_ds)

    predictions = trainer.predict(test_ds)
    preds = predictions.predictions.argmax(-1)
    labels = predictions.label_ids

    correct_predictions = (preds == labels).sum()
    total_samples = len(labels)

    cm = confusion_matrix(labels, preds)
    class_report = classification_report(labels, preds, target_names=class_names, output_dict=True)

    test_metrics.update({
        "detailed_metrics": class_report,
        "total_samples": total_samples,
        "correct_predictions": int(correct_predictions),
        "confusion_matrix": cm.tolist()
    })

    # Save confusion matrix
    plt = plot_confusion_matrix(cm, class_names)
    cm_path = f"{language}_gpt4o_confusion_matrix.png"
    plt.savefig(cm_path)
    plt.close()
    print(f"Saved confusion matrix to {cm_path}")

    # Save JSON results
    out_file = f"{language}_gpt4o_results.json"
    with open(out_file, "w") as f:
        json.dump(test_metrics, f, indent=2)
    print(f"Saved results to {out_file}")

    # Clean temp outputs
    temp_path = f"./results_{language}_gpt4o_temp"
    if os.path.exists(temp_path):
        shutil.rmtree(temp_path)
        print(f"Cleaned up {temp_path}")

In [ ]:
finetune_sentiment_model("french")

In [ ]:
finetune_sentiment_model("german")

In [ ]:
finetune_sentiment_model("spanish")

In [ ]:
finetune_sentiment_model("italian")

In [ ]:
finetune_sentiment_model("arabic")

In [ ]:
finetune_sentiment_model("hindi")

In [ ]:
finetune_sentiment_model("portuguese")